In [ ]:
import numpy as np 
import pandas as pd 
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

# TODO
- See if ticket and cabin have any importance
- Try data imputation instead of fillna

# Dataset

In [ ]:
df = pd.read_csv('/home/arjun/Desktop/GitHub/Kaggle-competitions/_Datasets/Titanic/train.csv')

In [ ]:
df.head()

In [ ]:
y = df['Survived']

In [ ]:
df = df.drop(['Name', 'Ticket', 'Cabin'], axis=1)

In [ ]:
df.head()

In [ ]:
df['Sex'] = df['Sex'].replace({'male': 0, 'female': 1})
df['Embarked'] = df['Embarked'].replace({'S':1, 'C':2, 'Q':3})

In [ ]:
df.fillna(df.mean(), inplace=True)
df.head()

In [ ]:
df_randomized = df.sample(frac=1).reset_index(drop=True)
train_df = df_randomized[:800]
test_df = df_randomized[800:]

In [ ]:
test_df

# Model Creation

In [ ]:
num_epochs = 5000
lr =.001
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self, inp_size, h1, h2, h3,  out_size):
        super(NeuralNet, self).__init__()
        self.inp_size = inp_size
        self.lay1 = nn.Linear(inp_size, h1)
        self.lay2 = nn.ReLU()
        self.lay3 = nn.Linear(h1, h2)
        self.lay4 = nn.ReLU()
        self.lay5 = nn.Linear(h2, h3)
        self.lay6 = nn.ReLU()
        self.lay7 = nn.Linear(h3, out_size)
        self.drop1 = nn.Dropout(0.5)
        self.drop2 = nn.Dropout(0.1)
        
    def forward(self,x):
        out = self.lay1(x)
        out = self.lay2(out)
        out = self.lay3(out)
        out = self.drop1(out)
        out = self.lay4(out)
        out = self.lay5(out)
        # out = self.drop2(out)
        out = self.lay6(out)
        out = self.lay7(out) # We don't apply sotmax the cross entropy loss will do that for us
        return out
    
model = NeuralNet(8,30,20,30,8).to(device)

In [ ]:
# class NeuralNet(nn.Module):
#     def __init__(self, inp_size, h1, h2, out_size):
#         super(NeuralNet, self).__init__()
#         self.fc1 = nn.Linear(inp_size, h1)
#         self.bn1 = nn.BatchNorm1d(h1)
#         self.drop1 = nn.Dropout(0.5)
#         self.fc2 = nn.Linear(h1, h2)
#         self.bn2 = nn.BatchNorm1d(h2)
#         self.drop2 = nn.Dropout(0.5)
#         self.fc3 = nn.Linear(h2, out_size)
        
#     def forward(self, x):
#         x = self.fc1(x)
#         x = self.bn1(x)
#         x = nn.ReLU()(x)
#         x = self.drop1(x)
#         x = self.fc2(x)
#         x = self.bn2(x)
#         x = nn.ReLU()(x)
#         x = self.drop2(x)
#         x = self.fc3(x)
#         return x

# model = NeuralNet(8, 30, 15, 2).to(device)
    

In [ ]:
lossCategory = nn.CrossEntropyLoss()
optimiser = torch.optim.Adam(model.parameters(), lr = lr)

# Training Model

In [ ]:
train_loader = DataLoader(dataset=torch.tensor(np.array(train_df)), shuffle=True, batch_size=1000)
test_loader = DataLoader(dataset=torch.tensor(np.array(test_df)), shuffle=True, batch_size=100)

In [ ]:
for epoch in range(num_epochs):
    avg_loss = 0
    for i,batch in enumerate(train_loader):

        y = batch[:,1].to(device).long()
        X =torch.cat((batch[:, :1], batch[:, 2:]), dim=1).to(device).to(torch.float32)

        # Forward pass
        output = model(X)
        
        # Backward pass
        loss = lossCategory(output, y)
        loss.backward()
        optimiser.step()
        optimiser.zero_grad()
        avg_loss += loss

    if not ((epoch+1)%100): print(f"Epoch:{epoch+1}/{num_epochs} Loss:{avg_loss/len(train_loader)}")

# Checking Accuracy

In [ ]:
correct = 0
total = 0
for i, batch in enumerate(train_loader):
    X = torch.cat((batch[:, :1], batch[:, 2:]), dim=1).to(device).to(torch.float32)
    y = batch[:, 1].to(device).long()
    output = model(X)
    softmax_output = F.softmax(output, dim=-1)
    _, y_pred = torch.max(softmax_output, dim=-1)
    correct += (y_pred == y).sum().item()
    total += y.size(0)
print('Accuracy:', correct / total * 100)
X.shape, y.shape, 

In [ ]:
correct = 0
total = 0
for i, batch in enumerate(test_loader):
    X = torch.cat((batch[:, :1], batch[:, 2:]), dim=1).to(device).to(torch.float32)
    y = batch[:, 1].to(device).long()
    output = model(X)
    softmax_output = F.softmax(output, dim=-1)
    _, y_pred = torch.max(softmax_output, dim=-1)
    correct += (y_pred == y).sum().item()
    total += y.size(0)
print('Accuracy:', correct / total * 100)


# Inference on Test Dataset

In [ ]:
df = pd.read_csv('/home/arjun/Desktop/GitHub/Kaggle-competitions/_Datasets/Titanic/test.csv')
df = df.drop(['Name', 'Ticket', 'Cabin'], axis=1)
df['Sex'] = df['Sex'].replace({'male': 0, 'female': 1})
df['Embarked'] = df['Embarked'].replace({'S':1, 'C':2, 'Q':3})
df.fillna(df.mean(), inplace=True)
ds = torch.tensor(np.array(df))
test_loader = torch.utils.data.DataLoader(dataset=ds, shuffle=False, batch_size=1)
ds.shape,df.shape

In [ ]:
l = []
id_ = []
for i,batch in enumerate(test_loader):
        X =batch.to(device).to(torch.float32)
        output = model(X)
        softmax_output = F.softmax(output, dim=-1)

        l.append(torch.argmax(softmax_output).item())
        id_.append(int(X[:,0].item()))

ans = pd.DataFrame({'PassengerId':id_,'Survived':l})
ans.to_csv('/home/arjun/Desktop/GitHub/Kaggle-competitions/_Datasets/Titanic/submission.csv', index=False)
print("Saved new O/P")
ans

# Best Score: 0.79